In [3]:
import pandas as pd
import numpy as np
import datetime as dt
import calendar
from dateutil.relativedelta import relativedelta



HHS_NHSN = pd.read_excel(r'C:\Users\ssing296\Desktop\Python-Coursera\HHS NHSN COVID Data.xlsx')
WOM_US_Rates = pd.read_excel(r'C:\Users\ssing296\Desktop\Python-Coursera\WOM US Rates Data.xlsx')

#WOM_US_Rates

n2w_dict = {0: 'none',1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five', \
             6: 'six', 7: 'seven', 8: 'eight', 9: 'nine'}

num2words = {1: 'One', 2: 'Two', 3: 'Three', 4: 'Four', 5: 'Five', \
             6: 'Six', 7: 'Seven', 8: 'Eight', 9: 'Nine', 10: 'Ten', \
            11: 'Eleven', 12: 'Twelve', 13: 'Thirteen', 14: 'Fourteen', \
            15: 'Fifteen', 16: 'Sixteen', 17: 'Seventeen', 18: 'Eighteen', \
            19: 'Nineteen', 20: 'Twenty', 30: 'Thirty', 40: 'Forty', \
            50: 'Fifty', 60: 'Sixty', 70: 'Seventy', 80: 'Eighty', \
            90: 'Ninety', 0: 'Zero'}


def n2w_mortality(n):
    try:
        return (n2w_dict[n])
    except KeyError:
        try:
            return n2w_dict[n-n%10] + n2w_dict[n%10].lower()
        except KeyError:
            return str(n)
        
def us_states_positivity_greater(df,threshold,ignore_state_list,ignore_states_flag):
    if ignore_states_flag == 1:
      df = df[~df['State'].isin(ignore_state_list)]
    else:
      print('No states removed!')
    
    

def hospital_bed(df,per_val,col_header):
    
    per_value = float(per_val)    
    
    hosp_states = df[df['Latest_Date_Rank'] == 1]['State'].iloc[0:50].tolist()
    hosp_values = df[df['Latest_Date_Rank'] == 1][col_header].iloc[0:50].tolist()

    #print(hosp_states)
    #print(hosp_values)
    
    hosp_dict = {}
    hosp_other_dict = {}
        
    for i in range(0,len(hosp_states)):
        if(round(hosp_values[i]*100,0) >= per_value):
            hosp_dict[hosp_states[i]] = hosp_values[i]
        else:    
            hosp_other_dict[hosp_states[i]] = hosp_values[i]
        
    print(len(hosp_other_dict))
    print(len(hosp_dict))
    print((hosp_dict))
    print(hosp_other_dict)

    hosp_dict = {k: v for k, v in sorted(hosp_dict.items(),reverse = True, key=lambda x: x[1])} 
    hosp_other_dict = {k: v for k, v in sorted(hosp_other_dict.items(),reverse = True, key=lambda x: x[1])} 
    
    hosp_dict = {key : round(hosp_dict[key]*100) for key in hosp_dict} 
    hosp_other_dict = {key : round(hosp_other_dict[key]*100) for key in hosp_other_dict} 
    
    expression = []
    
    for i in range(0, len(hosp_dict),1):
        expression.append(list(hosp_dict)[i])
        expression.append((list(hosp_dict.values())[i]))
        
    #print(expression)
    
    print_text = ''
    
    if(col_header == 'Percentage of Inpatient Beds Occupied Estimated'):
        print_text = 'Inpatient'
    else:
        print_text = 'ICU'
        
    if len(hosp_dict) == 0:
        
        a = 'No State had an ICU bed occupancy greater than 90%.'
    
    if len(hosp_dict) == 1:
        #state1 = list(hosp_dict)[0]
        #state1_val = round(list(hosp_dict.values())[0]*100)
      
        comment = 'One State had an  ' + print_text + ' bed occupancy greater than  ' + str(round(per_value)) + '%: {} ({:.0f}%)'.format(*expression)
        a = comment.format(*expression)
    
    elif len(hosp_dict) == 2:
        
        comment = 'Two States had an  ' + print_text + ' bed occupancy greater than ' + str(round(per_value)) + '%: {} ({:.0f}%) and {} ({:.0f}%)'.format(*expression)
        a = comment.format(*expression)
        
    elif len(hosp_dict) > 2:
        comment_length = len(hosp_dict) - 2
        #comment  = ' ' + str(len(hosp_dict)) + ' ({:.1f}% vs. {:.1f}% previous month)' + comment_length*(', {} ({:.1f}% vs. {:.1f}%)') + ' and {} ({:.1f}% vs. {:.1f}%) had 7-day average positivity rates greater than ' + str(threshold) + '%.'.format(*expression)
        
        comment = n2w_mortality(len(hosp_dict)).title()+' States had an ' + print_text + ' bed occupancy greater than ' + str(round(per_val)) + '%: ' + '{} ({:.0f})%' + comment_length*(', {} ({:.0f}%)') + ' and {} ({:.0f}%)' + str("") + ''.format(*expression)
        a = comment.format(*expression)
    
    if len(hosp_dict) == 1:
        hosp_other_states1 = list(hosp_other_dict)[0]
        hosp_other_states1_val = round(list(hosp_other_dict.values())[0])

        hosp_other_states2 = list(hosp_other_dict)[1]
        hosp_other_states2_val = round(list(hosp_other_dict.values())[1])

        hosp_other_states3 = list(hosp_other_dict)[2]
        hosp_other_states3_val = round(list(hosp_other_dict.values())[2])

        hosp_other_states4 = list(hosp_other_dict)[3]
        hosp_other_states4_val = round(list(hosp_other_dict.values())[3])
        b = ' with the next four highest States being: {} ({:.0f}%), {} ({:.0f}%),{} ({:.0f}%) and {} ({:.0f}%).'.format(hosp_other_states1,hosp_other_states1_val,hosp_other_states2,hosp_other_states2_val,hosp_other_states3,hosp_other_states3_val,hosp_other_states4,hosp_other_states4_val)
        
    elif len(hosp_dict) == 2:
        hosp_other_states1 = list(hosp_other_dict)[0]
        hosp_other_states1_val = round(list(hosp_other_dict.values())[0])

        hosp_other_states2 = list(hosp_other_dict)[1]
        hosp_other_states2_val = round(list(hosp_other_dict.values())[1])

        hosp_other_states3 = list(hosp_other_dict)[2]
        hosp_other_states3_val = round(list(hosp_other_dict.values())[2])

        b = ' with the next three highest States being: {} ({:.0f}%),{} ({:.0f}%) and {} ({:.0f}%).'.format(hosp_other_states1,hosp_other_states1_val,hosp_other_states2,hosp_other_states2_val,hosp_other_states3,hosp_other_states3_val)

    elif len(hosp_dict) == 3:
        hosp_other_states1 = list(hosp_other_dict)[0]
        hosp_other_states1_val = round(list(hosp_other_dict.values())[0])

        hosp_other_states2 = list(hosp_other_dict)[1]
        hosp_other_states2_val = round(list(hosp_other_dict.values())[1])

        b = ' with the next two highest States being: {} ({:.0f}%) and {} ({:.0f}%).'.format(hosp_other_states1,hosp_other_states1_val,hosp_other_states2,hosp_other_states2_val)

    elif len(hosp_dict) == 4:
        hosp_other_states1 = list(hosp_other_dict)[0]
        hosp_other_states1_val = round(list(hosp_other_dict.values())[0])


        b = ' with the next highest State being: {} ({:.0f}%).'.format(hosp_other_states1,hosp_other_states1_val)

    else:
        b = ''

    return a+b

        
        
#States_Comments_Hospital_Inpatient_Bed = hospital_bed(HHS_NHSN,85,'Percentage of Inpatient Beds Occupied Estimated')
States_Comments_Hospital_ICU_Bed = hospital_bed(HHS_NHSN,90,'Percentage of ICU Beds Occupied Estimated')



        
#print(States_Comments_Hospital_Inpatient_Bed)
print(States_Comments_Hospital_ICU_Bed)

50
0
{}
{'AK': 0.6385000000000001, 'AL': 0.7807, 'AR': 0.7452, 'AZ': 0.5287, 'CA': 0.7212999999999999, 'CO': 0.6463, 'CT': 0.5496, 'DC': 0.8558, 'DE': 0.8011, 'FL': 0.7698, 'GA': 0.8107, 'HI': 0.6537000000000001, 'IA': 0.6511, 'ID': 0.6325, 'IL': 0.5966, 'IN': 0.6409, 'KS': 0.7212000000000001, 'KY': 0.6008, 'LA': 0.6275, 'MA': 0.7254999999999999, 'MD': 0.7415999999999999, 'ME': 0.7121, 'MI': 0.7889, 'MN': 0.6944, 'MO': 0.7908, 'MS': 0.7120000000000001, 'MT': 0.5227, 'NC': 0.7504000000000001, 'ND': 0.5365, 'NE': 0.5915, 'NH': 0.6099, 'NJ': 0.4612, 'NM': 0.6288, 'NV': 0.6987000000000001, 'NY': 0.6755, 'OH': 0.6396999999999999, 'OK': 0.75, 'OR': 0.6416, 'PA': 0.7518, 'RI': 0.85, 'SC': 0.7136, 'SD': 0.5045000000000001, 'TN': 0.6743000000000001, 'TX': 0.7815000000000001, 'UT': 0.6227, 'VA': 0.6731, 'VT': 0.6804000000000001, 'WA': 0.6695, 'WI': 0.5061, 'WV': 0.7045999999999999}
No State had an ICU bed occupancy greater than 90%.
